In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df1=pd.read_csv("proteintry.csv")

df1.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Sugar Crops,Sugar & Sweeteners,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Miscellaneous,Obesity,Undernourished,Recovery_rate_percentage
0,Afghanistan,0.0000,9.7523,0.0277,0.0,35.9771,0.4067,0.0647,0.5824,3.1337,...,0.0,0.0000,0.1387,40.2477,0.0000,1.1370,0.0462,4.5,29.8,86.801012
1,Albania,0.1840,27.7469,0.0711,0.0,14.2331,1.8069,0.6274,1.2757,7.6582,...,0.0,0.0042,0.2677,22.2552,0.0084,3.2456,0.0544,22.3,6.2,60.413005
2,Algeria,0.0323,13.8360,0.0054,0.0,26.5633,1.2916,0.6350,1.1624,3.5088,...,0.0,0.0000,0.2745,36.1694,0.0269,3.1267,0.1399,26.6,3.9,68.425558
3,Angola,0.6285,15.2311,0.0277,0.0,20.3882,0.1756,5.4436,1.2754,7.6248,...,0.0,0.0092,0.0092,34.7782,0.0092,0.8133,0.0924,6.8,25.0,92.089522
4,Antigua and Barbuda,0.1535,33.1901,0.1289,0.0,10.5108,0.4850,8.2146,1.2586,16.0670,...,0.0,0.0000,0.0737,16.8161,0.0430,1.6024,0.2947,19.1,NaN,64.930556


In [3]:
df = df1.dropna()

In [4]:

df.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Sugar Crops,Sugar & Sweeteners,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Miscellaneous,Obesity,Undernourished,Recovery_rate_percentage
0,Afghanistan,0.0000,9.7523,0.0277,0.0,35.9771,0.4067,0.0647,0.5824,3.1337,...,0.0,0.0000,0.1387,40.2477,0.0000,1.1370,0.0462,4.5,29.8,86.801012
1,Albania,0.1840,27.7469,0.0711,0.0,14.2331,1.8069,0.6274,1.2757,7.6582,...,0.0,0.0042,0.2677,22.2552,0.0084,3.2456,0.0544,22.3,6.2,60.413005
2,Algeria,0.0323,13.8360,0.0054,0.0,26.5633,1.2916,0.6350,1.1624,3.5088,...,0.0,0.0000,0.2745,36.1694,0.0269,3.1267,0.1399,26.6,3.9,68.425558
3,Angola,0.6285,15.2311,0.0277,0.0,20.3882,0.1756,5.4436,1.2754,7.6248,...,0.0,0.0092,0.0092,34.7782,0.0092,0.8133,0.0924,6.8,25.0,92.089522
5,Argentina,0.1704,31.9799,0.0097,0.0,13.6702,2.0593,1.0223,0.5209,21.6250,...,0.0,0.0049,0.0438,18.0176,0.0000,1.0516,0.0000,28.5,4.6,89.647840


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
label_encoder=preprocessing.LabelEncoder()
df['Country']=label_encoder.fit_transform(df['Country'])


<ipython-input-6-a9035670ea0f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country']=label_encoder.fit_transform(df['Country'])


In [7]:
y=df.Recovery_rate_percentage
x=df.drop('Recovery_rate_percentage',axis=1)
np.random.seed(42)
x_shadow=x.apply(np.random.permutation)
x_shadow.columns={'shadow_'+ feat for feat in x.columns}
x_boruta=pd.concat([x,x_shadow],axis=1)

In [8]:
from sklearn.ensemble import RandomForestRegressor
forest=RandomForestRegressor(max_depth=5,random_state=42)
forest.fit(x_boruta,y)
feat_imp_x=forest.feature_importances_[:len(x.columns)]
feat_imp_shadow=forest.feature_importances_[len(x.columns):]
hits=feat_imp_x>feat_imp_shadow.max()

In [9]:
hits = np.zeros((len(x.columns)))
### repeat 70 times
for iter_ in range(70):
   ### make X_shadow by randomly permuting each column of X
   np.random.seed(iter_)
   x_shadow = x.apply(np.random.permutation)
   x_boruta = pd.concat([x, x_shadow], axis = 1)
   ### fit a random forest (suggested max_depth between 3 and 7)
   forest = RandomForestRegressor(max_depth = 5, random_state = 42)
   forest.fit(x_boruta, y)
   ### store feature importance
   feat_imp_x = forest.feature_importances_[:len(x.columns)]
   feat_imp_shadow = forest.feature_importances_[len(x.columns):]
   ### compute hits for this trial and add to counter
   hits += (feat_imp_x > feat_imp_shadow.max())

In [10]:
print(hits)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0. 12.  0.  5.  0.  0.]


In [11]:
df.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Sugar Crops,Sugar & Sweeteners,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Miscellaneous,Obesity,Undernourished,Recovery_rate_percentage
0,0,0.0000,9.7523,0.0277,0.0,35.9771,0.4067,0.0647,0.5824,3.1337,...,0.0,0.0000,0.1387,40.2477,0.0000,1.1370,0.0462,4.5,29.8,86.801012
1,1,0.1840,27.7469,0.0711,0.0,14.2331,1.8069,0.6274,1.2757,7.6582,...,0.0,0.0042,0.2677,22.2552,0.0084,3.2456,0.0544,22.3,6.2,60.413005
2,2,0.0323,13.8360,0.0054,0.0,26.5633,1.2916,0.6350,1.1624,3.5088,...,0.0,0.0000,0.2745,36.1694,0.0269,3.1267,0.1399,26.6,3.9,68.425558
3,3,0.6285,15.2311,0.0277,0.0,20.3882,0.1756,5.4436,1.2754,7.6248,...,0.0,0.0092,0.0092,34.7782,0.0092,0.8133,0.0924,6.8,25.0,92.089522
5,4,0.1704,31.9799,0.0097,0.0,13.6702,2.0593,1.0223,0.5209,21.6250,...,0.0,0.0049,0.0438,18.0176,0.0000,1.0516,0.0000,28.5,4.6,89.647840
